## Train part

In [ ]:
!pip3 install ultralytics

In [ ]:
!cp /content/drive/MyDrive/Skoltech_Dead_Electronics.v1i.yolov8.zip .
!unzip Skoltech_Dead_Electronics.v1i.yolov8.zip

In [ ]:
# Train
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
results = model.train(data="/content/data.yaml", epochs=70, batch=64, imgsz=640, save_period=10, patience=20)

## Export part

In [ ]:
!apt install python3.10-venv

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!git clone https://github.com/airockchip/ultralytics_yolov8
%cd ultralytics_yolov8
!python3 -m venv env
!source env/bin/activate
!pip3 uninstall ultralytics -y
!pip3 install onnx onnxruntime

In [ ]:
!PYTHONPATH=./ python ./ultralytics/engine/exporter.py

In [ ]:
!cp /content/runs/detect/train3/weights/best.onnx /content/model_ver_0.0.1_70_epochs.onnx

In [ ]:
!python3 --version

In [ ]:
%cd /content
!wget https://raw.githubusercontent.com/airockchip/rknn-toolkit2/refs/heads/master/rknn-toolkit2/packages/x86_64/rknn_toolkit2-2.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip3 install rknn_toolkit2-2.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

In [ ]:
%cd /content
!git clone https://github.com/airockchip/rknn_model_zoo
%cd rknn_model_zoo/examples/yolov8

In [ ]:
import os
import cv2
from tqdm import tqdm

# Resize image to another for calib
RESIZE_TO_IMGSZ = None # None if no resize needed

IMG_PATH = "/content/train/images"
files = os.listdir(IMG_PATH)

with open("/content/data_subset.txt", "w") as fd:
  for i in tqdm(files):
    print(os.path.join(IMG_PATH, i), file=fd)
    if RESIZE_TO_IMGSZ:
      img = cv2.imread(os.path.join(IMG_PATH, i))
      img = cv2.resize(img, (RESIZE_TO_IMGSZ, RESIZE_TO_IMGSZ))
      cv2.imwrite(os.path.join(IMG_PATH, i), img)

In [ ]:
%cd python

In [ ]:
!python3 convert.py /content/model_ver_0.0.1_70_epochs.onnx rk3588 i8

In [ ]:
!cp ../model/yolov8.rknn /content/model_ver_0.0.1_70_epochs.rknn